# V1 

## TensorFlow - Convolutional Neural Networks

#### Two types of deep neural networks:
<ul><li>
    Convolutional Neural Networks
</li><li>
    Recurrent Neural Networks
</li></ul>

Convolution Neural Network (CNN) are designed to process data through multiple layers of arrays.  Convolutional neural network uses three basic ideas −
<ul>
    <li>Local respective fields</li>
    <li>Convolution</li>
    <li>Pooling</li>
</ul>

## TensorFlow Implementation of CNN

### Step 1 - 
Import the necissary modules of TensorFlow and data

In [9]:
import tensorflow as tf
import numpy as np

#from tensorflow.examples.tutorials.mnist import input_data ## Old code
import tensorflow_datasets as tfds

### Step 2 - 
Declare a function called run_cnn(), which includes various parameters and optimization variables with declaration of data placeholders. These optimization variables will declare the training pattern.

In [10]:
# Construct a tf.data.Dataset
#dataset = tfds.load(name="mnist", split=tfds.Split.TRAIN)

def run_cnn():
    #mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)
    mnist = tfds.load(name="mnist", split=tfds.Split.TRAIN)
    learning_rate = 0.0001
    epochs = 10
    batch_size = 50

### Step 3

Declare the training data placeholders of 28px by 28px.

In [11]:
# Use the t1 compat to make this V1 code still work
tf.compat.v1.disable_eager_execution() # Disable V2's eager execution

#x = tf.placeholder(tf.float32, [None, 784])
x = tf.compat.v1.placeholder(tf.float32, [None, 784])
x_shaped = tf.reshape(x, [-1, 28, 28, 1])
#y = tf.placeholder(tf.float32, [None, 10])
y = tf.compat.v1.placeholder(tf.float32, [None, 10])

### Step 4 − 

Now it is important to create some convolutional layers −

In [12]:
def create_new_conv_layer(
    input_data, num_input_channels, num_filters,filter_shape, pool_shape, name):
    
    conv_filt_shape = [
        filter_shape[0], filter_shape[1], num_input_channels, num_filters]
    
    #weights = tf.Variable(tf.truncated_normal(conv_filt_shape, stddev = 0.03), name = name+'_W')
    weights = tf.Variable(tf.random.truncated_normal(conv_filt_shape, stddev = 0.03), name = name+'_W')
    #bias = tf.Variable(tf.truncated_normal([num_filters]), name = name+'_b')
    bias = tf.Variable(tf.random.truncated_normal([num_filters]), name = name+'_b')
    
    #Out layer defines the output
    out_layer = tf.nn.conv2d(input_data, weights, [1, 1, 1, 1], padding = 'SAME')
    
    out_layer += bias
    out_layer = tf.nn.relu(out_layer)
    ksize = [1, pool_shape[0], pool_shape[1], 1]
    strides = [1, 2, 2, 1]
    out_layer = tf.nn.max_pool(
        out_layer, ksize = ksize, strides = strides, padding = 'SAME')
    
    return out_layer

layer1 = create_new_conv_layer(x_shaped, 1, 32, [5, 5], [2, 2], name = 'layer1')
layer2 = create_new_conv_layer(layer1, 32, 64, [5, 5], [2, 2], name = 'layer2')

### Step 5 − 

Let us flatten the output ready for the fully connected output stage

In [13]:
flattened = tf.reshape(layer2, [-1, 7 * 7 * 64])

#wd1 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 1000], stddev = 0.03), name = 'wd1')
wd1 = tf.Variable(tf.random.truncated_normal([7 * 7 * 64, 1000], stddev = 0.03), name = 'wd1')
#bd1 = tf.Variable(tf.truncated_normal([1000], stddev = 0.01), name = 'bd1')
bd1 = tf.Variable(tf.random.truncated_normal([1000], stddev = 0.01), name = 'bd1')

dense_layer1 = tf.matmul(flattened, wd1) + bd1
dense_layer1 = tf.nn.relu(dense_layer1)

### Step 6 - 

Add another layer with specific softmax activations with the required optimizer defines the accuracy assessment

In [16]:
## I'm not sure why they defined these variables in a function, so we couldn't use them later.
learning_rate = 0.0001
epochs = 10
batch_size = 50

#wd2 = tf.Variable(tf.truncated_normal([1000, 10], stddev = 0.03), name = 'wd2')
wd2 = tf.Variable(tf.random.truncated_normal([1000, 10], stddev = 0.03), name = 'wd2')
#bd2 = tf.Variable(tf.truncated_normal([10], stddev = 0.01), name = 'bd2')
bd2 = tf.Variable(tf.random.truncated_normal([10], stddev = 0.01), name = 'bd2')

dense_layer2 = tf.matmul(dense_layer1, wd2) + bd2
y_ = tf.nn.softmax(dense_layer2)

cross_entropy = tf.reduce_mean(
   tf.nn.softmax_cross_entropy_with_logits(logits = dense_layer2, labels = y))

optimiser = tf.compat.v1.train.AdamOptimizer(learning_rate = learning_rate).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init_op = tf.compat.v1.global_variables_initializer()

### Step 7 -

Run our bad code

In [23]:
tf.summary.scalar('accuracy', accuracy)
merged = tf.compat.v1.summary.merge_all()
writer = tf.compat.v1.summary.FileWriter('E:\TensorFlowProject')

with tf.compat.v1.Session() as sess:
    sess.run(init_op)
    total_batch = int(len(mnist.train.labels) / batch_size)
  
    for epoch in range(epochs):
        avg_cost = 0
    for i in range(total_batch):
        batch_x, batch_y = mnist.train.next_batch(batch_size = batch_size)
        _, c = sess.run([optimiser, cross_entropy], feed_dict = {x:batch_x, y: batch_y})
        avg_cost += c / total_batch
    test_acc = sess.run(accuracy, feed_dict = {x: mnist.test.images, y: mnist.test.labels})
    summary = sess.run(merged, feed_dict = {x: mnist.test.images, y:mnist.test.labels})
    writer.add_summary(summary, epoch)

print("\nTraining complete!")
writer.add_graph(sess.graph)
print(sess.run(accuracy, feed_dict = {x: mnist.test.images, y: mnist.test.labels}))

def create_new_conv_layer(
    input_data, num_input_channels, num_filters,filter_shape, pool_shape, name):
    
    conv_filt_shape = [filter_shape[0], filter_shape[1], num_input_channels, num_filters]

    weights = tf.Variable(tf.truncated_normal(conv_filt_shape, stddev = 0.03), name = name+'_W')
    bias = tf.Variable(tf.truncated_normal([num_filters]), name = name+'_b')

    #Out layer defines the output
    out_layer = tf.nn.conv2d(input_data, weights, [1, 1, 1, 1], padding = 'SAME')

    out_layer += bias
    out_layer = tf.nn.relu(out_layer)
    ksize = [1, pool_shape[0], pool_shape[1], 1]
    strides = [1, 2, 2, 1]
    out_layer = tf.nn.max_pool(out_layer, ksize = ksize, strides = strides, padding = 'SAME')

    return out_layer

if __name__ == "__main__":
    run_cnn()

2024-06-20 03:24:27.153022: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-06-20 03:24:27.153061: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: 23d480978687
2024-06-20 03:24:27.153069: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: 23d480978687
2024-06-20 03:24:27.153144: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 545.23.6
2024-06-20 03:24:27.153169: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 470.239.6
2024-06-20 03:24:27.153175: E external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:244] kernel version 470.239.6 does not match DSO version 545.23.6 -- cannot find working devices in this configuration
2024-06-20 03:24:27.161866: I tensorflow/compiler/mlir/m

NameError: name 'mnist' is not defined